In [16]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import os
print(os.listdir("./cell_images/"))
from keras.models import Sequential


['Parasitized', 'Uninfected']


In [17]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.optimizers import SGD

In [18]:
classifier = Sequential()

# Step 1 - Convolution
classifier.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))

# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Adding a second convolutional layer
classifier.add(Conv2D(32, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Step 3 - Flattening
classifier.add(Flatten())

# Step 4 - Full connection
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dropout(rate=0.5))
classifier.add(Dense(units = 1, activation = 'sigmoid'))

# # Compiling the CNN

classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

Showing a sample image

In [19]:
from keras.preprocessing.image import ImageDataGenerator

In [20]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   validation_split=0.3,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
train_set = train_datagen.flow_from_directory('./cell_images/',
                                                 target_size = (64, 64),
                                                 batch_size =32,
                                                 class_mode = 'binary',
                                                 subset='training')
test_set = train_datagen.flow_from_directory('./cell_images/',
                                                 target_size = (64, 64),
                                                 batch_size =32,
                                                 class_mode = 'binary',
                                                 subset='validation')

Found 19292 images belonging to 2 classes.
Found 8266 images belonging to 2 classes.


In [7]:
classifier.fit_generator(train_set,
                           steps_per_epoch=345,
                           epochs=10,
                           validation_data=test_set,
                           validation_steps=86)

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
345/345 [==============================] - 55s 159ms/step - loss: 0.6549 - acc: 0.6144 - val_loss: 0.6398 - val_acc: 0.6512
Epoch 2/10
345/345 [==============================] - 49s 142ms/step - loss: 0.3932 - acc: 0.8478 - val_loss: 0.3256 - val_acc: 0.8390
Epoch 3/10
345/345 [==============================] - 52s 151ms/step - loss: 0.2613 - acc: 0.9026 - val_loss: 0.2011 - val_acc: 0.9132
Epoch 4/10
345/345 [==============================] - 50s 146ms/step - loss: 0.2179 - acc: 0.9211 - val_loss: 0.2210 - val_acc: 0.9059
Epoch 5/10
345/345 [==============================] - 49s 143ms/step - loss: 0.2235 - acc: 0.9197 - val_loss: 0.1870 - val_acc: 0.9172
Epoch 6/10
345/345 [==============================] - 49s 143ms/step - loss: 0.1928 - acc: 0.9326 - val_loss: 0.1650 - val_acc: 0.9288
Epoch 7/10
345/345 [==============================] - 53s 152ms/step - loss: 0.1909 - acc: 0.9369 - val_loss: 0.1741 - val_acc: 0.9253
Epoch 8

saving the model

In [11]:
model_json = classifier.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

# serialize weights to HDF5
classifier.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


Loading

In [14]:
from keras.models import model_from_json
# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

Loaded model from disk


In [21]:
loaded_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
score = loaded_model.evaluate_generator(generator=test_set,)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

ValueError: `steps=None` is only valid for a generator based on the `keras.utils.Sequence` class. Please specify `steps` or use the `keras.utils.Sequence` class.

In [ ]:
hist = classifier.history.history
hist.keys()
val_acc=hist['val_acc']
val_loss=hist['val_loss']
acc=hist['acc']
loss=hist['loss']

In [ ]:
epochs=range(1,len(acc)+1)
plt.plot(epochs,acc,'bo',label='Training_Accuracy')
plt.plot(epochs,val_acc,'b',label='Validation_Accuracy')
plt.figure()
plt.show()